In [ ]:
#!/usr/bin/env python
from datetime import datetime
from grovepi import *
import grovepi
import time, math
from threading import Thread

def LED_Bewegung():
    pir_sensor = 8
    led = 3
    pinMode(pir_sensor, "INPUT")
    pinMode(led, "OUTPUT")
    time.sleep(1)
    while True:
        try:
            if digitalRead(pir_sensor):
                print 'Motion Detected'
                print 'LED ON!'
                digitalWrite(led, 1)
                time.sleep(.5)
            else:
                digitalWrite(led, 0)
                print("LED OFF!")
                print '-'
            time.sleep(1)
        except IOError:
            print ("Error")
        except KeyboardInterrupt:
            digitalWrite(led, 0)
            break

def Geraeuschsensor():
    print("Geraeuschsensor misst")
    loudness_sensor = 0

    while True:
        try:
            global Lautstaerke
            Lautstaerke = grovepi.analogRead(loudness_sensor)
            print("sensor_value = %d" % Lautstaerke)
            time.sleep(.25)

            if Lautstaerke > 100:
                Lautstaerke = True
                break
            elif Lautstaerke < 100:
                Lautstaerke = False
            print(Lautstaerke)

        except IOError:
            print ("Error")

def Initialise():
    Geraeuschsensor()

def Uhrzeit():
    global Stunde, Minute
    now = datetime.now()
    Stunde = int(now.hour)
    Minute = int(now.minute)

def Datum():
    global Tag, Monat, Jahr
    now = datetime.now()
    Tag = int(now.strftime("%d"))
    Monat = int(now.strftime("%m"))
    Jahr = int(now.year)

def Temperaturabfrage():
    print("Temperatursensor misst")

    sensor = 4

    blue = 0
    white = 1

    [temp, humidity] = grovepi.dht(sensor, white)

    if math.isnan(temp) == False and \
            math.isnan(humidity) == False:
        global aktTemp
        aktTemp = int(float("%.02f" % (temp)))

def Displayan():
    global grovepi, display

    display = 5
    grovepi.pinMode(display, "OUTPUT")

    grovepi.fourDigit_init(display)
    time.sleep(.5)

    grovepi.fourDigit_brightness(display, 0)
    time.sleep(.5)

def Displayaus():
    grovepi.fourDigit_off(display)

def Display_Uhrzeit():
    Uhrzeit()
    print("Display zeigt Uhrzeit: ")
    print(Stunde, Minute)
    grovepi.fourDigit_score(display, Stunde, Minute)
    time.sleep(3)
    Displayaus()
    time.sleep(1)

def Display_Datum():
    Datum()
    print("Display zeigt Datum: ")
    print(Tag, Monat)
    print(Jahr)
    leading_zero = 0
    grovepi.fourDigit_score(display, Tag, Monat)
    time.sleep(3)
    grovepi.fourDigit_number(display, Jahr, leading_zero)
    time.sleep(3)
    Displayaus()
    time.sleep(1)

def Display_Temperatur():
    Temperaturabfrage()
    print("Display zeigt Temperatur: ")
    print(aktTemp)
    leading_zero = 1
    grovepi.fourDigit_number(display, aktTemp, leading_zero)
    time.sleep(3)
    Displayaus()
    time.sleep(1)

def Display_Text():
    grovepi.fourDigit_segment(display, 0, 57)  # 57 = C
    grovepi.fourDigit_segment(display, 1, 63)  # 63 = O
    grovepi.fourDigit_segment(display, 2, 63)  # 63 = O
    grovepi.fourDigit_segment(display, 3, 56)  # 56 = L
    time.sleep(3)
    grovepi.fourDigit_segment(display, 0, 118)  # 118 = H
    grovepi.fourDigit_segment(display, 1, 0x6d)  # 0x6d= S
    grovepi.fourDigit_segment(display, 2, 56)  # 56 = L
    grovepi.fourDigit_segment(display, 3, 0x3e)  # 0x3e= U
    time.sleep(3)
    Displayaus()

def Neustart():
    global start_time
    start_time = time.time()
    Geraeuschsensor()

def Ablauf():
    start="Ja"
    global start_time
    start_time=time.time()
    global counter
    counter=0
    while start=="Ja":
        try:
            while (time.time()-start_time<4) and Lautstaerke==True \
                    or (time.time()-start_time<4 and digitalRead(pir_sensor)):
                time.sleep(1)
                if counter==0:
                    Display_Uhrzeit()
                    counter=1
                    Neustart()
                    continue
                if counter==1:
                    Display_Datum()
                    counter=2
                    Neustart()
                    continue
                if counter==2:
                    Display_Temperatur()
                    counter=3
                    Neustart()
                    continue
                if counter==3:
                    Display_Text()
                    counter=0
                    Neustart()
                    time.sleep(1)
                    Neustart()
                    continue
            counter = 0
            start_time = time.time()
            time.sleep(.5)

        except KeyboardInterrupt:
            grovepi.fourDigit_off(display)

Displayan()
if KeyboardInterrupt:
    grovepi.fourDigit_off(display)

Initialise()

if __name__=='__main__':
    Thread(target=LED_Bewegung).start()
    Thread(target=Ablauf).start()